Данный ноутбук является частью проекта "Рекомендательные системы с элементами NLP" команды МОВС ВШЭ

In [1]:
import pandas as pd
import numpy as np

# Загружаем данные

In [2]:
# Загрузим в ноутбук
!gdown 1I_uFqSR7TiGp7BfKH6wwnpmiBk1ZAzww
!gdown 1CP4deOmpaOqnwdnqZCcZBqpZjBxKDs6l
!gdown 1S0Wz20_d-HV9USfxwqmE4FRtkOovs2XK

Downloading...
From: https://drive.google.com/uc?id=1I_uFqSR7TiGp7BfKH6wwnpmiBk1ZAzww
To: /content/items.csv
100% 31.8M/31.8M [00:00<00:00, 44.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CP4deOmpaOqnwdnqZCcZBqpZjBxKDs6l
To: /content/users.csv
100% 29.9M/29.9M [00:00<00:00, 63.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S0Wz20_d-HV9USfxwqmE4FRtkOovs2XK
To: /content/interactions.csv
100% 180M/180M [00:02<00:00, 75.1MB/s]


In [8]:
items_df = pd.read_csv('/content/items.csv')
users_df = pd.read_csv('/content/users.csv')
interactions_df = pd.read_csv('/content/interactions.csv')

In [24]:
interactions_df['last_watch_dt'] = pd.to_datetime(interactions_df['last_watch_dt'])

In [183]:
class PopularityModel():
    def __init__(self, n: int):
        self.n = n
        self.top_n = []

    def fit(self, df: pd.DataFrame):
        self.top_n = df.loc[:, 'item_id'].value_counts()[:self.n].index.tolist()

    def predict(self):
        predicts = self.top_n
        return predicts

In [184]:
def split_df(df, n_days):
  test_df = df[df['last_watch_dt'] >= df['last_watch_dt'].max() - pd.DateOffset(days=n_days)].copy()
  train_df = df[df['last_watch_dt'] < df['last_watch_dt'].max() - pd.DateOffset(days=n_days)].copy()
  return train_df, test_df

In [265]:
train, test = split_df(interactions_df, 7)
model = PopularityModel(n=10)
model.fit(train)
preds = model.predict()

In [256]:
test_users = pd.DataFrame({'user_id': test['user_id'].unique()})
test_users['predict'] = test_users.apply(lambda x: model.predict(x), axis=1)

In [266]:
def map_at_k(interactions, predicts, k):
  map_list = []
  for user in predicts['user_id'].values:
    user_interactions = interactions[interactions['user_id'] == user]
    user_predicts = predicts[predicts['user_id'] == user]
    max_k = min(len(user_interactions), k)
    user_precision = []
    for i in range(1, max_k + 1):
      precision = len(set(user_predicts['predict'].values[0][:i])&set(user_interactions['item_id'].values[:i]))/i
      user_precision.append(precision)
    map_list.append(np.mean(user_precision))
  return np.mean(map_list)

In [267]:
map_at_10 = map_at_k(test, test_users, 10)

In [268]:
print(f'MAP@10: {map_at_10}')

MAP@10: 0.06577649275764488
